# Chapter 4: Expecting the Unexpected
## Raising Exceptions
#### Friday, January 18th
By Ellen Considine

* In Python, "error" and "exception" are used almost interchangeably.
* Common exceptions include
    * __SyntaxError__
    * __ZeroDivisionError__
    * __IndexError__
    * __TypeError__
    * __AttributeError__
    * __KeyError__
    * __NameError__




## Raising an exception

In [16]:
class EvenOnly(list):
    def append(self, integer):
        if not isinstance(integer, int):
            raise TypeError("Only integers can be added")
        if integer % 2:
            raise ValueError("Only even numbers can be added")
        super().append(integer)
            

In [17]:
e = EvenOnly()
e.append("a string")

TypeError: Only integers can be added

In [18]:
e.append(3)

ValueError: Only even numbers can be added

In [19]:
e.append(4) #everything's fine!


Note: we could still get other values into the list by using index or slice notation... This could be avoided by overriding other methods in the __list__ class.

## The effects of an exception

Once an exception is raised, nothing else executes...

In [20]:
def no_return():
    print("About to raise exception...")
    raise Exception("This is always raised")
    print("This line will never be executed")
    return("This will never be returned")

In [21]:
no_return()

About to raise exception...


Exception: This is always raised

The same is true for a function that calls another function that raises an exception:

In [25]:
def call_exceptor():
    print("This is the outer function")
    no_return()
    print("an exception was raised, so this doesn't run")

In [26]:
call_exceptor()

This is the outer function
About to raise exception...


Exception: This is always raised

## Handling exceptions
It turns out that exceptions can be handled at any level after they are raised.

Here's the most basic syntax:

In [28]:
try:
    no_return()
except: 
    print("I caught an exception!")
print("executed after the exception")

About to raise exception...
I caught an exception!
executed after the exception


We can specify one or more exceptions:

In [33]:
def fun_division(anumber):
    try:
        if anumber == 13:
            raise ValueError("13 is an unlucky number")
        return 100/anumber
    except (ZeroDivisionError, TypeError):
        return "Enter a number other than zero"
    

In [34]:
for val in (0, "hello", 50, 13):
    print(fun_division(val))

Enter a number other than zero
Enter a number other than zero
2.0


ValueError: 13 is an unlucky number

We can also stack them:

In [68]:
def fun_division2(anumber):
    try:
        if anumber == 13:
            raise ValueError("13 is an unlucky number")
        return 100/anumber
    except ZeroDivisionError:
        return "Enter a number other than zero"
    except TypeError:
        return "Enter a numerical value"
    except ValueError:
        print("Not 13!")
#         raise
    print("handled")
    

In [69]:
for val in (0, "hello", 50, 13):
    print(fun_division2(val))

Enter a number other than zero
Enter a numerical value
2.0
Not 13!
handled
None


Because only the first matching clause will be run, we want to stack exceptions from least to most general.

In [38]:
def fun_division3(anumber):
    try:
        return 100/anumber
    except TypeError:
        return "Enter a numerical value"
    except Exception as e:
        return e
    

Sometimes when we catch an exception we need a reference to the Exception object:

In [41]:
try:
    raise ValueError("This is an argument")
except ValueError as e:
    print("The exception arguments were", e.args)

The exception arguments were ('This is an argument',)


Two more keywords, __else__ and __finally__, allow us even more flexibility when handling exceptions.

In [79]:
import random
some_exceptions = [ValueError, TypeError, IndexError, None]

try:
    choice = random.choice(some_exceptions)
    print("raising {}".format(choice))
    if choice:
        raise choice("An error")
except ValueError:
    print("Caught a ValueError")
except Exception as e:
    print("Caught some other error: %s" % (e.__class__.__name__))
else:
    print("There was no error")
finally:
    print("This code is always called")


raising None
There was no error
This code is always called


Notes on the __finally__ clause:
* Common useful examples include:
    * cleaning up an open database connection
    * closing an open file
    * sending a closing handshake over the network
* Handle will be executed before the return statement in the try clause is executed

Notes on the __else__ clause:
* Instead, we could put the "no error" code after the __try-except__ block
* The difference is that the __else__ block will not be executed if an exception is caught and handled

## The exception hierarchy

* Most exceptions (but not all) are subclasses of the __Exception__ class
* __Exception__ inherits from the __BaseException__ class
* All exceptions must extend the __BaseException__ class or one of its subclasses

Two exceptions, __SystemExit__ and __KeyboardInterrupt__, derive directly from __BaseException__ instead of __Exception__.
* The first is called when the program exits naturally (we called "sys exit" somewhere in the code, we closed the window, etc.)
* The second is called when we explicitly interrupt program execution with a key combination (normally, _Ctrl + C_)
* Both include cleanup code in a __finally__ clause, so we generally don't need to handle them explicitly

* Using __except Exception: __ will catch all exceptions other than __SystemExit__ and __KeyboardInterrupt__
* Using __except: __ will catch all subclasses of __BaseException__
    * Still, it's better to use __except BaseException: __ so other people know you are intentionally handling the special case exceptions as well as the regular Exception subclasses

## Defining our own exceptions

It's easy to define new exceptions! All we have to do is inherit from the __Exception__ class:

In [61]:
class InvalidWithdrawal(Exception):
    pass

raise InvalidWithdrawal("You don't have $50 in your account")

InvalidWithdrawal: You don't have $50 in your account

Passing arguments into the exception:

* Often a string message is used
* Any number of arguments is allowed
* __Exception.\_\_init\_\_ __ method accepts arguments and stores them as a tuple in an attribute called args


Customizing the initializer:
    

In [62]:
class InvalidWithdrawal(Exception):
    def __init__(self, balance, amount):
        super().__init__("account doesn't have ${}".format(amount))
        self.amount = amount
        self.balance = balance
    def overage(self):
        return self.amount - self.balance
    
raise InvalidWithdrawal(25, 50)

InvalidWithdrawal: account doesn't have $50

Here's how we would handle an __InvalidWithdrawal__ exception if one was raised:

In [64]:
try:
    raise InvalidWithdrawal(25, 50)
except InvalidWithdrawal as e:
    print("I'm sorry, your withdrawal is more than your balance by ${}".format(e.overage()))

I'm sorry, your withdrawal is more than your balance by $25


### General thoughts
* Exceptions are not a bad thing that we should try to avoid
* Exception syntax is effective for flow control: decision making, branching, and message passing
    * Alternative to an __if__ statement
* Use exceptions even when the circumstances are only a little bit exceptional

__Case Study__ on pages 114 - 123 of the book shows how exceptions are used in the larger context of objects, inheritance, and modules.

### Takeaways from discussion in OOP group

* This has lots of potential in user-oriented coding (software development) in addition to error handling in "getting stuff done" coding
* In statistical simulation, this kind of error handling allows us to run lots of simulations and handle errors so they don't crash the whole system 
* __assess__ clauses are also useful for checking whether the data is in the right format, etc.
* __try-except__ blocks are pretty efficient compared to __if-else__ blocks 
    * __try__ doesn't take any computation whereas an __if__ uses a logic check